# Data Preprocessing

## Import Libraries

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, monotonically_increasing_id, lit, date_add, explode, sequence
from pyspark.sql.functions import corr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Initialize Spark session

In [2]:
sc = SparkContext(master = 'local')
spark = SparkSession.builder \
          .appName("Python Spark Data Preprocessing") \
          .getOrCreate()

25/01/03 20:00:04 WARN Utils: Your hostname, Khim3 resolves to a loopback address: 127.0.1.1; using 192.168.75.108 instead (on interface wlo1)
25/01/03 20:00:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/03 20:00:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read and Load data

In [3]:
# Load the data
df = spark.read.csv('../data/NFLX.csv', header=True, inferSchema=True)
df.show(5)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2018-02-05|     262.0|267.899994|250.029999|254.259995|254.259995|11896100|
|2018-02-06|247.699997|266.700012|     245.0|265.720001|265.720001|12595800|
|2018-02-07|266.579987|272.450012|264.329987|264.559998|264.559998| 8981500|
|2018-02-08|267.079987|267.619995|     250.0|250.100006|250.100006| 9306700|
|2018-02-09|253.850006|255.800003|236.110001|249.470001|249.470001|16906900|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



## Exploratory Data Analysis (EDA)

### Check null values, schema, and remove duplicates

In [4]:
# 1. Check for null values
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

# 2. Get schema and row/column counts
df.printSchema()
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

# 3. Drop duplicates
df = df.dropDuplicates()

+----+----+----+---+-----+---------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|
+----+----+----+---+-----+---------+------+
|   0|   0|   0|  0|    0|        0|     0|
+----+----+----+---+-----+---------+------+

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

Number of rows: 1009
Number of columns: 7


In [5]:
df.show(10)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2018-08-08|352.209991|352.290009|346.609985|347.609985|347.609985| 5394700|
|2018-09-11|344.670013|356.149994|343.899994|355.929993|355.929993| 6198100|
|2019-07-09|379.059998| 384.76001|     377.5|379.929993|379.929993| 6932800|
|2020-01-15|338.679993|343.170013|336.600006|339.070007|339.070007| 5158000|
|2020-08-10|493.350006|497.459991|478.630005|483.380005|483.380005| 4691200|
|2021-06-08|     497.0|498.820007|489.369995|492.390015|492.390015| 2374000|
|2019-01-07|302.100006|316.799988|301.649994|315.339996|315.339996|18620100|
|2019-06-10|363.649994|367.100006|349.290009| 352.01001| 352.01001| 7810300|
|2020-07-28|496.019989|497.790009| 487.76001| 488.51001| 488.51001| 5986700|
|2020-08-31|521.159973| 531.98999|517.580017|529.559998|529.559998| 4941400|

### Check the data distribution

In [6]:
df.describe().show()

25/01/03 20:00:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|              Open|              High|               Low|             Close|         Adj Close|           Volume|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|              1009|              1009|              1009|              1009|              1009|             1009|
|   mean|419.05967286224006|425.32070308027704|412.37404380178464|419.00073292071335|419.00073292071335| 7570685.03468781|
| stddev|108.53753170401455|109.26295957119457|107.55586739006048|108.28999877034995|108.28999877034995|5465535.225689977|
|    min|        233.919998|        250.649994|        231.229996|        233.880005|        233.880005|          1144000|
|    max|        692.349976|         700.98999|        686.090027|        691.690002|        691.690002|         58904300|
+-------+-------

### Check correlation between features

In [7]:
# List of columns to calculate correlation with 'Close'
columns_to_corr = [col for col in df.columns if col not in ["Close", "Adj Close", "Date"]]

# Calculate correlations and show results
correlations = [
    df.select(corr("Close", col).alias(f"Close_{col}_Corr")).collect()[0][0] 
    for col in columns_to_corr
]

# Display correlations
for col, corr_value in zip(columns_to_corr, correlations):
    print(f"Correlation between Close and {col}: {corr_value}")

Correlation between Close and Open: 0.9968121105128466
Correlation between Close and High: 0.9985508166555647
Correlation between Close and Low: 0.9985438753668088
Correlation between Close and Volume: -0.4133618746720189


## **The dataset is pretty clean and could be used for modelling**

## Create train and test datasets

In [8]:
# order the dates in ascending order
df = df.orderBy('Date')
# Calculate split index
split_index = int(df.count() * 0.8)

# Split the dataset into training and testing sets
train = df.limit(split_index)  # Take the first 80% of rows
test = df.subtract(train)      # Subtract the training set from the original DataFrame to get the test set
test_copy = test.select("*") 
# Display row counts of the resulting DataFrames to verify the split
print(f"Training set row count: {train.count()}")
print(f"Testing set row count: {test.count()}")
test.show(5)

Training set row count: 807
Testing set row count: 202
+----------+----------+----------+----------+----------+----------+-------+
|      Date|      Open|      High|       Low|     Close| Adj Close| Volume|
+----------+----------+----------+----------+----------+----------+-------+
|2021-06-08|     497.0|498.820007|489.369995|492.390015|492.390015|2374000|
|2021-10-05|606.940002|640.390015|606.890015|634.809998|634.809998|9534300|
|2021-10-22|651.809998|665.460022|651.809998|664.780029|664.780029|6186000|
|2021-11-08|650.289978|     656.0|643.789978|651.450012|651.450012|2887500|
|2021-11-17|     690.0| 700.98999|686.090027|691.690002|691.690002|2732800|
+----------+----------+----------+----------+----------+----------+-------+
only showing top 5 rows



## Save the test datasets

In [9]:
test_pandas = test.toPandas()

# Convert the Date column to datetime format for accurate sorting
test_pandas["Date"] = pd.to_datetime(test_pandas["Date"])

# Sort the Pandas DataFrame by the Date column
test_pandas = test_pandas.sort_values(by="Date")
test_pandas.head()
test_pandas.to_csv("test.csv", index=False)

25/01/03 20:00:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
